In [1]:
!pip install scikit-learn==1.5.0

In [2]:
import pandas as pd
import pickle
from pathlib import Path
from datetime import datetime
from dateutil.relativedelta import relativedelta

from sklearn.metrics import mean_squared_error

In [3]:
year = 2023
month = 3

In [4]:
# Generate input and output file names
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = Path(f'./output/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [5]:
# Load the model
with open('homework/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
# Load data
df = pd.read_parquet(input_file)

# Preprocess
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [7]:
# Prepare dicts for model
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Calculate metrics
print(f"Standard deviation of predictions: {round(y_pred.std(), 2)}")
print(f"Mean predicted duration: {round(y_pred.mean(), 2)}")
from math import sqrt

print(f"RMSE: {round(sqrt(mean_squared_error(df['duration'], y_pred)), 2)}")

Standard deviation of predictions: 6.25
Mean predicted duration: 14.2
RMSE: 8.25


**2: Preparing the output**

In [8]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [9]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [11]:
import os

output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

output_file = 'output/predictions.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
import os

file_size = os.path.getsize(output_file) / (1024 * 1024)  # Convert bytes to MB
print(f"{file_size:.0f}M")

65M


**3. Creating the scoring script**

In [13]:
!jupyter nbconvert homework4.ipynb --to script

[NbConvertApp] Converting notebook homework4.ipynb to script
[NbConvertApp] Writing 2047 bytes to homework4.py


**4. Virtual environment. Hash for Scikit-Learn**

In [15]:
import json
with open('Pipfile.lock') as g:
    j = json.loads(g.read())
    print (j['default']['scikit-learn']['hashes'] [0])

sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


**5. Parametrize the script**

In [2]:
!python homework4.py --year=2023 --month=4

Standard deviation of predictions: 6.35
Mean predicted duration: 14.29
RMSE: 8.44
63M


**6. Docker container**

In [6]:
!docker run ride-duration-predictor --year=2023 --month=5

Standard deviation of predictions: 1.39
Mean predicted duration: 0.19
RMSE: 19.34
67M
